In [1]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(1)

In [2]:
# Dependecies
%matplotlib inline
import matplotlib.pyplot as plt
import joblib
import numpy as np
import pandas as pd
import sklearn
import sklearn.datasets

In [3]:
# Read the billboard data into pandas
df2 = pd.read_csv('data/grouped_audio_billboard_genre_data.csv')

# Drop the null columns where all values are null
df2 = df2.dropna(axis='columns', how='all')

# Drop the null rows
df2 = df2.dropna()
df2.head()

# Display dataframe 
df2.head()

,track_id,genre,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,energy,key,loudness,mode,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,week_pos,instance,peak_pos,weeks_on_chart
0,"AdictoTainy, Anuel AA & Ozuna",pop,0.0,270740.0,91.0,0.734,0.836,10.0,-4.803,0.0,...,0.01700,0.000016,0.1790,0.623,80.002,4.0,98.0,1.0,86.0,2.0
1,The Ones That Didn't Make It Back HomeJustin M...,country,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.0,...,0.00000,0.000000,0.0000,0.000,0.000,0.0,100.0,1.0,100.0,1.0
2,ShallowLady Gaga & Bradley Cooper,pop,0.0,215733.0,89.0,0.572,0.385,7.0,-6.362,1.0,...,0.37100,0.000000,0.2310,0.323,95.799,4.0,28.0,1.0,28.0,1.0
3,EnemiesPost Malone Featuring DaBaby,rap,1.0,196760.0,86.0,0.542,0.674,6.0,-4.169,1.0,...,0.05880,0.000000,0.0955,0.667,76.388,4.0,98.0,1.0,16.0,15.0
4,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo",rap,1.0,228185.0,61.0,0.948,0.623,8.0,-5.725,0.0,...,0.00124,0.000001,0.0716,0.856,135.979,4.0,94.0,1.0,94.0,3.0


In [4]:
# Assigning numbers to genre
df2['genre'] = df2['genre'].replace('pop',float(0)).replace('country',float(1))\
.replace('hiphop',float(2)).replace('other',float(3)).replace('latin',float(3)).replace('latin',float(4))\
.replace('house',float(5)).replace('folk',float(6)).replace('r&b',float(7)).replace('adult standards',float(8))\
.replace('rock',float(9)).replace('metal',float(10)).replace('show tunes',float(11)).replace('soul',float(12))\
.replace('rap',float(13)).replace('jazz',float(14))

In [5]:
# Discretize variable into equal-sized buckets
df2['target'] = pd.qcut(df2['peak_pos'],10, labels = False)

## Create a Train Test Split

Use `target`for the y values

In [6]:
from sklearn.model_selection import train_test_split
y = pd.Series(df2["target"])
X = df2.drop(columns=["track_id", "target" ,'peak_pos'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [7]:
y.describe()

count    28375.000000
mean         4.339912
std          2.802909
min          0.000000
25%          2.000000
50%          4.000000
75%          7.000000
max          9.000000
Name: target, dtype: float64

In [8]:
X_train

,genre,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,week_pos,instance,weeks_on_chart
11734,9.0,0.0,245640.0,77.0,0.817,0.599,0.0,-9.249,0.0,0.0328,0.1320,0.000311,0.0873,0.548,108.873,4.0,73.0,1.0,1.0
28040,9.0,0.0,258399.0,63.0,0.569,0.744,8.0,-6.480,1.0,0.0298,0.0559,0.000003,0.0915,0.655,96.596,4.0,34.0,1.0,1.0
12045,3.0,0.0,240866.0,61.0,0.634,0.677,5.0,-7.278,0.0,0.0304,0.0117,0.001030,0.1260,0.558,135.012,4.0,67.0,1.0,1.0
14139,9.0,0.0,132280.0,0.0,0.714,0.464,8.0,-11.080,1.0,0.0514,0.9000,0.283000,0.1030,0.921,124.553,4.0,80.0,1.0,2.0
23223,8.0,0.0,135506.0,53.0,0.570,0.259,7.0,-14.393,1.0,0.0332,0.8050,0.000000,0.1120,0.485,124.863,4.0,100.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10955,14.0,0.0,189946.0,38.0,0.706,0.467,7.0,-9.523,1.0,0.0852,0.7130,0.000000,0.2530,0.735,107.053,4.0,63.0,1.0,1.0
17289,14.0,0.0,151866.0,7.0,0.652,0.656,7.0,-7.156,1.0,0.0967,0.2590,0.050900,0.1420,0.771,161.895,4.0,92.0,1.0,1.0
5192,9.0,0.0,162986.0,19.0,0.587,0.649,9.0,-9.411,1.0,0.0321,0.6050,0.000006,0.1390,0.831,96.111,4.0,89.0,1.0,1.0
12172,9.0,0.0,233506.0,63.0,0.626,0.565,9.0,-10.027,1.0,0.0271,0.1240,0.000000,0.0811,0.924,116.610,4.0,65.0,1.0,1.0


In [9]:
y_train

11734    2
28040    0
12045    1
14139    2
23223    9
        ..
10955    1
17289    6
5192     5
12172    1
235      0
Name: target, Length: 21281, dtype: int64

## Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)


X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



In [11]:
from pickle import dump
dump(X_scaler, open('scaler.pkl', 'wb'))

## Create a Deep Learning Model

In [12]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense

In [13]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [14]:
y_train_categorical

array([[0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [15]:
X_train_scaled.shape

(21281, 19)

In [16]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=10, activation='relu', input_dim=19))
model.add(Dense(units=10, activation='softmax'))

In [17]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                200       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
Total params: 310
Trainable params: 310
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=400,
    shuffle=True,
    verbose=2
)

Epoch 1/400
666/666 - 2s - loss: 2.2735 - accuracy: 0.1443
Epoch 2/400
666/666 - 2s - loss: 2.1420 - accuracy: 0.2052
Epoch 3/400
666/666 - 2s - loss: 1.9902 - accuracy: 0.2570
Epoch 4/400
666/666 - 2s - loss: 1.8813 - accuracy: 0.3025
Epoch 5/400
666/666 - 1s - loss: 1.7980 - accuracy: 0.3293
Epoch 6/400
666/666 - 1s - loss: 1.7304 - accuracy: 0.3628
Epoch 7/400
666/666 - 1s - loss: 1.6731 - accuracy: 0.3929
Epoch 8/400
666/666 - 1s - loss: 1.6229 - accuracy: 0.4203
Epoch 9/400
666/666 - 1s - loss: 1.5797 - accuracy: 0.4488
Epoch 10/400
666/666 - 1s - loss: 1.5408 - accuracy: 0.4682
Epoch 11/400
666/666 - 1s - loss: 1.5060 - accuracy: 0.4882
Epoch 12/400
666/666 - 1s - loss: 1.4745 - accuracy: 0.4987
Epoch 13/400
666/666 - 1s - loss: 1.4461 - accuracy: 0.5168
Epoch 14/400
666/666 - 2s - loss: 1.4196 - accuracy: 0.5347
Epoch 15/400
666/666 - 2s - loss: 1.3958 - accuracy: 0.5514
Epoch 16/400
666/666 - 2s - loss: 1.3734 - accuracy: 0.5614
Epoch 17/400
666/666 - 1s - loss: 1.3522 - accura

666/666 - 1s - loss: 0.7082 - accuracy: 0.8614
Epoch 138/400
666/666 - 1s - loss: 0.7075 - accuracy: 0.8609
Epoch 139/400
666/666 - 1s - loss: 0.7051 - accuracy: 0.8649
Epoch 140/400
666/666 - 1s - loss: 0.7040 - accuracy: 0.8619
Epoch 141/400
666/666 - 1s - loss: 0.7021 - accuracy: 0.8611
Epoch 142/400
666/666 - 1s - loss: 0.7016 - accuracy: 0.8638
Epoch 143/400
666/666 - 1s - loss: 0.6996 - accuracy: 0.8642
Epoch 144/400
666/666 - 1s - loss: 0.6987 - accuracy: 0.8630
Epoch 145/400
666/666 - 1s - loss: 0.6969 - accuracy: 0.8651
Epoch 146/400
666/666 - 1s - loss: 0.6966 - accuracy: 0.8666
Epoch 147/400
666/666 - 1s - loss: 0.6939 - accuracy: 0.8661
Epoch 148/400
666/666 - 1s - loss: 0.6934 - accuracy: 0.8673
Epoch 149/400
666/666 - 1s - loss: 0.6920 - accuracy: 0.8674
Epoch 150/400
666/666 - 1s - loss: 0.6912 - accuracy: 0.8664
Epoch 151/400
666/666 - 1s - loss: 0.6900 - accuracy: 0.8676
Epoch 152/400
666/666 - 1s - loss: 0.6883 - accuracy: 0.8664
Epoch 153/400
666/666 - 1s - loss: 0.6

Epoch 272/400
666/666 - 1s - loss: 0.6057 - accuracy: 0.8938
Epoch 273/400
666/666 - 1s - loss: 0.6051 - accuracy: 0.8943
Epoch 274/400
666/666 - 1s - loss: 0.6047 - accuracy: 0.8955
Epoch 275/400
666/666 - 1s - loss: 0.6048 - accuracy: 0.8927
Epoch 276/400
666/666 - 1s - loss: 0.6042 - accuracy: 0.8930
Epoch 277/400
666/666 - 1s - loss: 0.6038 - accuracy: 0.8946
Epoch 278/400
666/666 - 1s - loss: 0.6031 - accuracy: 0.8948
Epoch 279/400
666/666 - 1s - loss: 0.6024 - accuracy: 0.8922
Epoch 280/400
666/666 - 1s - loss: 0.6058 - accuracy: 0.8903
Epoch 281/400
666/666 - 1s - loss: 0.6010 - accuracy: 0.8941
Epoch 282/400
666/666 - 1s - loss: 0.6027 - accuracy: 0.8938
Epoch 283/400
666/666 - 1s - loss: 0.6012 - accuracy: 0.8977
Epoch 284/400
666/666 - 1s - loss: 0.6009 - accuracy: 0.8964
Epoch 285/400
666/666 - 1s - loss: 0.6012 - accuracy: 0.8953
Epoch 286/400
666/666 - 1s - loss: 0.6012 - accuracy: 0.8947
Epoch 287/400
666/666 - 1s - loss: 0.6014 - accuracy: 0.8937
Epoch 288/400
666/666 - 

## Quantify our Trained Model

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

## Save the Model

In [ ]:
# Save the model
model.save('Models/billb_spot_DL.h5')

##  Loading our Model

In [ ]:
# Load the model
from tensorflow.keras.models import load_model
billboard_spotify_model = load_model('Models/billb_spot_DL.h5')

###  Evaluating the loaded model

In [ ]:
model_loss, model_accuracy = billboard_spotify_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")